# PPFNet  
요한  
미완성

In [18]:
import os
import torch
import mediapipe as mp
import numpy as np
import cv2

## Load Data

In [19]:
# shape 확인
print(torch.load("./output/tensor/KETI_SL_0000000002/left_hand.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/right_hand.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/pose.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/face.pt").shape)

torch.Size([160, 21, 3])
torch.Size([160, 21, 3])
torch.Size([160, 33, 3])
torch.Size([160, 468, 3])


shape : [a, b, c]  
a : time step  
b : point#  
c : x, y, z 좌표

In [20]:
left_hand = torch.load("./output/tensor/KETI_SL_0000000002/left_hand.pt")
right_hand = torch.load("./output/tensor/KETI_SL_0000000002/right_hand.pt")
pose = torch.load("./output/tensor/KETI_SL_0000000002/pose.pt")
#torch.load("./output/tensor/KETI_SL_0000000002/face.pt")

In [21]:
# position data
print(right_hand[0].shape)
print(right_hand[1].shape)
print(right_hand[2].shape)

torch.Size([21, 3])
torch.Size([21, 3])
torch.Size([21, 3])


## PPFConv  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.PPFConv

아직 다운샘플링 적용안함. 추후에 해야함.

In [24]:
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import PPFConv
from torch_cluster import knn_graph, fps
from torch_geometric.nn import global_max_pool


class PPFNet(torch.nn.Module):
    def __init__(self):
        super(PPFNet, self).__init__()

        torch.manual_seed(12345)

        # hidden features: 0
        # 4D descriptor: 4
        mlp1 = Sequential(Linear(0+4, 32),
                          ReLU(),
                          Linear(32, 48),
                          ReLU(),
                          Linear(48, 32))
        self.conv1 = PPFConv(mlp1)

        # hidden features: 32
        # 4D descriptor: 4
        mlp2 = Sequential(Linear(32+4, 48),
                          ReLU(),
                          Linear(48, 48),
                          ReLU(),
                          Linear(48, 32))
        self.conv2 = PPFConv(mlp2)

        # hidden features: 32
        # 4D descriptor: 4
        mlp3 = Sequential(Linear(32+4, 48),
                          ReLU(),
                          Linear(48, 48),
                          ReLU(),
                          Linear(48, 32))
        self.conv3 = PPFConv(mlp3)
        
#         classification이 아닌 transformer를 위한 임베딩
#         num_classes = 10
#         self.classifier = Linear(32, num_classes)
        
    def forward(self, pos, normal, batch):
        edge_index = knn_graph(pos, k=16, batch=batch, loop=False)
        
        x = self.conv1(x=None, pos=pos, normal=normal, edge_index=edge_index)
        x = x.relu()
        x = self.conv2(x=x, pos=pos, normal=normal, edge_index=edge_index)
        x = x.relu()
        x = self.conv3(x=x, pos=pos, normal=normal, edge_index=edge_index)
        x = x.relu()

        x = global_max_pool(x, batch)  # [num_examples, hidden_channels]

        return x  # 32차원 임베딩 벡터
#         return self.classifier(x)


model = PPFNet()
print(model)

PPFNet(
  (conv1): PPFConv(local_nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=48, bias=True)
    (3): ReLU()
    (4): Linear(in_features=48, out_features=32, bias=True)
  ), global_nn=None)
  (conv2): PPFConv(local_nn=Sequential(
    (0): Linear(in_features=36, out_features=48, bias=True)
    (1): ReLU()
    (2): Linear(in_features=48, out_features=48, bias=True)
    (3): ReLU()
    (4): Linear(in_features=48, out_features=32, bias=True)
  ), global_nn=None)
  (conv3): PPFConv(local_nn=Sequential(
    (0): Linear(in_features=36, out_features=48, bias=True)
    (1): ReLU()
    (2): Linear(in_features=48, out_features=48, bias=True)
    (3): ReLU()
    (4): Linear(in_features=48, out_features=32, bias=True)
  ), global_nn=None)
)


In [28]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(right_hand, batch_size=10, shuffle=False)  # shuffle X 중요!
# test_loader = DataLoader(test_dataset, batch_size=10)

print(train_loader.dataset.shape)

model = PPFNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.

def train(model, optimizer, loader):
    model.train()
    
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()  # Clear gradients.
        logits = model(data.pos, data.batch)  # Forward pass.
        loss = criterion(logits, data.y)  # Loss computation.
        loss.backward()  # Backward pass.
        optimizer.step()  # Update model parameters.
        total_loss += loss.item() * data.num_graphs

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(model, loader):
    model.eval()

    total_correct = 0
    for data in loader:
        logits = model(data.pos, data.batch)
        pred = logits.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())

    return total_correct / len(loader.dataset)

torch.Size([160, 21, 3])


일단 normal vector는 임의로 설정  
추후 이전 관절과의 변위를 normal vector로 설정

batch도 일단 None  
설정할 방법 생각해보기